In [ ]:
%matplotlib inline

import sys

import matplotlib.pyplot as plt

import os

import matplotlib as mpl

import numpy as np

from PIL import Image

from pyquaternion import Quaternion

import scipy

In [ ]:
import sklearn

In [ ]:
from scipy.optimize import linear_sum_assignment

In [ ]:
sys.path.append("/home/nils/data_fusion/nuscenes-devkit/python-sdk")

In [ ]:
from nuscenes.nuscenes import NuScenes

nusc = NuScenes(version='v1.0-mini', dataroot='nuscenes', verbose=True)

In [ ]:
from nuscenes.utils.data_classes import LidarPointCloud

from nuscenes.utils.geometry_utils import view_points


In [ ]:
import sklearn.cluster

# Task 1

## Picking a random sample

In [ ]:
def rotate_pointcloud(cloud, quaternion):
    def rot(an_array):
        return quaternion.rotate(an_array)
    return np.array(list(map(rot, cloud)))

In [ ]:
def create_bounding_box(annotation_token):
    '''
    creates bounding box used for plotting
    '''
    van_annotation = nusc.get('sample_annotation', annotation_token)
    relativeboxvectors = [(van_annotation['size'][1]/2,van_annotation['size'][0]/2, 0),
                         (-van_annotation['size'][1]/2,van_annotation['size'][0]/2, 0),
                         (-van_annotation['size'][1]/2,-van_annotation['size'][0]/2, 0),
                         (van_annotation['size'][1]/2,-van_annotation['size'][0]/2, 0),
                         (van_annotation['size'][1]/2,van_annotation['size'][0]/2, 0)]
    return np.array(relativeboxvectors), van_annotation['translation']

In [ ]:
my_sample = nusc.sample[10]
my_scene_token =my_sample['scene_token']
my_scene = nusc.get('scene', my_scene_token)
first_sample_token = my_scene['first_sample_token']

In [ ]:
#create list of sample tokens (of whole scene)
current_sample_token = first_sample_token
sample_tokens = []
while current_sample_token != '':
    sample_tokens.append(current_sample_token)
    #next annotaton token
    current_sample_token = nusc.get('sample', current_sample_token)['next']

In [ ]:
def remove_clutter(sample_token):
    '''
    returns all points inside vehicle bounding boxes
    and the number of vehicle bounding boxes that actually contain measurements
    '''
    current_sample_from_scene = nusc.get('sample', sample_token)

    pointsensor_token = current_sample_from_scene['data']['LIDAR_TOP']
    pcl_path = nusc.get('sample_data', pointsensor_token)['filename']
    pc =LidarPointCloud.from_file(os.path.join('nuscenes',pcl_path))
    points = pc.points
    
    ego_pose = nusc.get('ego_pose', nusc.get('sample_data', pointsensor_token)['ego_pose_token'])
    

    calibrated_sensor_token = nusc.get('sample_data', pointsensor_token)['calibrated_sensor_token']
    calibrated_sensor = nusc.get('calibrated_sensor', calibrated_sensor_token)
    pointsensor_quaternion = Quaternion(calibrated_sensor['rotation'])
    ego_quaternion = Quaternion(ego_pose['rotation'])

    sensor_rotated_points = np.dot(pointsensor_quaternion.rotation_matrix, points[:3,:])
    rotated_points = np.dot(ego_quaternion.rotation_matrix, sensor_rotated_points)

    # use broadcasting to add translation to x and y dimension
    absolute_points = rotated_points+np.array(ego_pose['translation'][:3]).reshape(-1,1)
    
    all_annotation_points = np.empty([2, 0])
    vehicle_counter = 0
    for i in range(len(current_sample_from_scene['anns'])):

        current_annotation_token = current_sample_from_scene['anns'][i]
        current_annotation = nusc.get('sample_annotation', current_annotation_token)
        if current_annotation['category_name'][:7] != 'vehicle':
            continue

        #print(f'progress: {i/len(current_sample_from_scene["anns"])}')
        ### rotate points by inverse bounding box rotation (translation to its reference frame unnecessary for only filtering)
        reverse_annotation_quaternion = Quaternion(w=-1*current_annotation['rotation'][0], x=current_annotation['rotation'][1], y=current_annotation['rotation'][2], z=current_annotation['rotation'][3])
        points_bounding_rotation = np.dot(reverse_annotation_quaternion.rotation_matrix, absolute_points)

        #new stuff
        relativeboxvectors, position = create_bounding_box(current_annotation_token)

        rotated_box = relativeboxvectors+reverse_annotation_quaternion.rotate(position)

        x_max = rotated_box[:,0].max()
        x_min = rotated_box[:,0].min()

        y_max = rotated_box[:,1].max()
        y_min = rotated_box[:,1].min()

        ## Remove points that are outside the bounding box
        mask = np.ones(points_bounding_rotation.shape[1], dtype=bool)

        mask = np.logical_and(mask, points_bounding_rotation[0] > x_min)
        mask = np.logical_and(mask, points_bounding_rotation[0] < x_max)
        mask = np.logical_and(mask, points_bounding_rotation[1] > y_min)
        mask = np.logical_and(mask, points_bounding_rotation[1] < y_max)

        points_filtered_rotated = points_bounding_rotation[:,mask]
        if len(points_filtered_rotated[0])>0:
            vehicle_counter += 1
        ### rotate points back
        annotation_quaternion = Quaternion(current_annotation['rotation'])
        points_filtered_absolute = np.dot(annotation_quaternion.rotation_matrix, points_filtered_rotated)

        all_annotation_points = np.concatenate((all_annotation_points,points_filtered_absolute[:2]),axis=1)
    return all_annotation_points, vehicle_counter

In [ ]:
ims = []
fig = plt.figure()
ax = fig.add_subplot(111)  # fig and axes created once
mean_positions_whole_scene = []
annotation_points_whole_scene = []
#clustering by kmeans
colors_whole_scene = []
# number of vehicles to cluster(only matters for kmeans)
vehicle_counters_whole_scene = []
mean_positions_whole_scene_dbscan = []
colors_dbscan = []
# go over all samples of our scene

for current_sample_token in sample_tokens:
    # remove clutter
    all_annotation_points, vehicle_counter = remove_clutter(current_sample_token)
    annotation_points_whole_scene.append(all_annotation_points)
    vehicle_counters_whole_scene.append(vehicle_counter)
    
    # apply clustering algorithms
    ## k_means
    clustered = sklearn.cluster.k_means(np.swapaxes(all_annotation_points,0,1), vehicle_counter)
    mean_positions = clustered[0]
    colors = clustered[1]
    colors_whole_scene.append(colors)
    mean_positions_whole_scene.append(mean_positions)
    
    ## dbscan
    db = sklearn.cluster.DBSCAN(eps=1.5, min_samples=1).fit(np.swapaxes(all_annotation_points,0,1))

    dbscan_clustered = [[] for x in range((db.labels_).max()+1)]

    for i,point in enumerate(db.labels_):
        dbscan_clustered[point].append(all_annotation_points[:,i])
    mean_positions = []
    for cluster in dbscan_clustered:
        means = np.array(cluster).mean(axis=0)
        mean_positions.append(means)
    mean_positions_whole_scene_dbscan.append(mean_positions)
    colors_dbscan.append(db.labels_)
    
    # add plot of mean positions to movie
    ims.append(plt.scatter(np.swapaxes(mean_positions,0,1)[0],np.swapaxes(mean_positions,0,1)[1], s=2,).findobj())
ax.axis('equal')
ani = mpl.animation.ArtistAnimation(fig, ims, repeat=False)
ani.save('annotations_im.gif', writer='pillow')

In [ ]:
def distance(p1, p2):
    return np.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

In [ ]:
#use coloring of 
#clustered with k-means: colors_whole_scene 
#or dbscan: colors_dbscan

f, ax = plt.subplots(4,sharex=True)
#ax.axis('equal')
i = 0
axes = plt.gca()
axes.set_xlim([380, 430])
axes.set_ylim([1100,1250])
ax[i].scatter(annotation_points_whole_scene[i][0], annotation_points_whole_scene[i][1], c=colors_dbscan[i], s=3)
vehicle_counters_whole_scene[i]

i = 1
ax[i].scatter(annotation_points_whole_scene[i][0], annotation_points_whole_scene[i][1], c=colors_dbscan[i], s=3)
vehicle_counters_whole_scene[i]

i = 2
ax[i].scatter(annotation_points_whole_scene[i][0], annotation_points_whole_scene[i][1], c=colors_dbscan[i], s=3)
vehicle_counters_whole_scene[i]

i = 3
ax[i].scatter(annotation_points_whole_scene[i][0], annotation_points_whole_scene[i][1], c=colors_dbscan[i], s=3)
vehicle_counters_whole_scene[i]

In [ ]:
#preprocessed_positions = mean_positions_whole_scene
preprocessed_positions = np.array(mean_positions_whole_scene_dbscan)
n = 0


# create cost matrix based on distances of cluster means
cost_matrix = np.zeros((len(preprocessed_positions[n]),len(preprocessed_positions[n+1])))
for i in range(len(preprocessed_positions[n])):
    for j in range(len(preprocessed_positions[n+1])):
        cost_matrix[i,j] = distance(preprocessed_positions[n][i],preprocessed_positions[n+1][j])

# algorithm to match 2 sets of points
row_ind, col_ind = linear_sum_assignment(cost_matrix)
for i in range(len(row_ind)):
    x = [preprocessed_positions[n][row_ind[i]][0],preprocessed_positions[n+1][col_ind[i]][0]]
    y = [preprocessed_positions[n][row_ind[i]][1],preprocessed_positions[n+1][col_ind[i]][1]]
    print(x)
    axes = plt.gca()
    axes.set_xlim([380, 430])
    axes.set_ylim([1100,1250])
    plt.plot(x,y)

# todo:

# apply kalmann filter to all clusters
## get initial velocity

## get initial positions

## make predictions

## compare predictions to cluster means of next time step

## throw out everything with velocity > 100km/h

# some plots

In [ ]:
i = 2
x = np.swapaxes(mean_positions_whole_scene_dbscan[i],0,1)[0]
y = np.swapaxes(mean_positions_whole_scene_dbscan[i],0,1)[1]
axes = plt.gca()
axes.set_xlim([380, 430])
axes.set_ylim([1100,1250])
plt.scatter(x,y, c='red', s=5)

In [ ]:
i = 2
x = np.swapaxes(mean_positions_whole_scene[i],0,1)[0]
y = np.swapaxes(mean_positions_whole_scene[i],0,1)[1]
axes = plt.gca()
axes.set_xlim([380, 430])
axes.set_ylim([1100,1250])
plt.scatter(x,y, c='red', s=5)

In [ ]:
plt.scatter(annotation_points_whole_scene[i][0], annotation_points_whole_scene[i][1], s=0.1,c=db.labels_)

In [ ]:
#k-means with clutter
a = sklearn.cluster.k_means(np.swapaxes(pc.points[:2],0,1), len(first_sample_from_scene['anns']))

plt.scatter(pc.points[0], pc.points[1], s=0.1,c=a[1])